## MTH377 - Convex Optimisation (COO) - Assessment 1

### Problem 3

- In this problem, we have defined set S as a subset of $R^n$ and is defined by a collection of inequalities:

- S = {x $\in \mathbb{R}^n$: for every $j = 1,.....,m$,   $g_j(x) \geq 0$}

- With any set S, we have defined a potential function:
$$
\Psi (x) = -\sum_{j=1}^{+m} \log g_{j}(x)
$$

- The analytical center is defined as the vector $x$ which minimises the potential function defined above.

- We have been given an example instance in $\mathbb{R}^2$ where we have been three inequalities as defined below:
$$
g_1(x_1,x_2) = 2x_2 - x_1
$$
$$
g_2(x_1,x_2) = 2x_1 - x_2
$$
$$
g_3(x_1,x_2) = 1 - x_2 - x_1
$$
- We need to compute the analytical center by framing this as an unconstrained optimisation problem with starting point as $(0.25, 0.25)$ and applying the combination descent algorithm.

- Our potential function would be of the form:
$$
\Psi (x_1,x_2) = \log (\frac {1}{(2x_2-x_1)(2x_1 - x_2)(1 - x_1 - x_2)})
$$

- We can take this as our $f(x_1, x_2)$ to be minimised

In [7]:
import numpy as np
import math

# Define the function
def f(x1,x2):
    return (-1)*(math.log((2*x2-x1)*(2*x1-x2)*(1-x1-x2)))

- For the gradient, we need expressions for the partial derivates of f(x,y) wrt x and y calculated below:
$$
\frac{\partial f}{\partial x_1} = \frac {1}{2x_2-x_1} + \frac {-2}{2x_1-x_2} + \frac {1}{1-x_1-x_2}

$$
$$
\frac{\partial f}{\partial x_2} = \frac {-2}{2x_2-x_1} + \frac {1}{2x_1-x_2} + \frac {1}{1-x_1-x_2}
$$

- Thus the gradient of f(x,y) is a vector of the form:

$$
\nabla f = \begin{vmatrix} 
\frac{\partial f}{\partial x} \\
\\
\frac{\partial f}{\partial y}
\end{vmatrix}
$$

In [8]:
# Define the gradient
def gradient_f(x1,x2):

    # Taking x1 as x and x2 as y
    df_dx = (1/(2*x2-x1)) + (-2/(2*x1-x2)) + (1/(1-x1-x2))
    df_dy = (-2/(2*x2-x1)) + (1/(2*x1-x2)) + (1/(1-x1-x2))

    # Return the gradient as a numpy array
    return np.array([df_dx,df_dy])

- For the Hessian Matrix, we need the double partial derivates with respect to x and y. Calculated them as follows:
$$ \frac{\partial^{2} f}{\partial {x_1}^2} = \frac {1}{(2x_2-x_1)^2} + \frac {4}{(2x_1-x_2)^2} + \frac {1}{(1-x_1-x_2)^2} $$
$$ \frac{\partial^{2} f}{\partial {{x_2}^2}} = \frac {4}{(2x_2-x_1)^2} + \frac {1}{(2x_1-x_2)^2} + \frac {1}{(1-x_1-x_2)^2} $$
$$ \frac{\partial^{2} f}{\partial {x_1x_2}} = \frac {-2}{(2x_2-x_1)^2} + \frac {-2}{(2x_1-x_2)^2} + \frac {1}{(1-x_1-x_2)^2} $$

- Thus the Hessian Matrix of the given function f(x,y) at any (x,y) will be:
$$
H(x_1,x_2) = \begin{vmatrix} 
\frac{\partial^{2} f}{\partial {x_1}^2}   \frac{\partial^{2} f}{\partial {x_1x_2}} \\
\\
\frac{\partial^{2} f}{\partial {x_2x_1}}   \frac{\partial^{2} f}{\partial {{x_2}^2}} 
\end{vmatrix}

In [9]:
# Define the Hessian
def hessian_f(x1, x2):

    # Taking x1 as x and x2 as y
    d2f_dx2 = (1/((2*x2-x1)**2)) + (4/((2*x1-x2)**2)) + (1/((1-x1-x2)**2))
    d2f_dy2 = (4/((2*x2-x1)**2)) + (1/((2*x1-x2)**2)) + (1/((1-x1-x2)**2))
    d2f_dxdy = (-2/((2*x2-x1)**2)) + (-2/((2*x1-x2)**2)) + (1/((1-x1-x2)**2))

     # Returning the Hessian as a 2x2 matrix
    return np.array([[d2f_dx2, d2f_dxdy], [d2f_dxdy, d2f_dy2]])

- We find the descent direction 'd' using the Backtracking-Linear Search Routine or the Armijo Rule.
- We check for the following inequality to be true:
$$
f(x) - f(x+td) < \alpha(-Df(x)·(td))
$$
where $\alpha$ is an acceptable descent parameter $\in (0,0.5)$

- We start with t = 1 and then till the about condition holds true, we multiply t with a reduction factor $\beta$ $\in (0,1)$

- I have implemented the same below. In our case $d$ would be a 1x2 matrix and $Df(x)$ becomes the gradient $\nabla f$ at the original point $x,y$.

- The tweak which we do here to prevent any error from coming up is that we additionally put the inequality conditions on the next $x_1,x_2$ coordinate by ensuring that the step size $t$ and direction $d$ ensures that the next point is in the permissiable region defined by:

$$
2x_2 - x_1 > 0
$$
$$
2x_1 - x_2 > 0
$$
$$
1 - x_2 - x_1 > 0
$$

In [10]:
# Defining the Backtracking Line Search
def linear_search(d, alpha, beta, point):
    t = 1
    while((f(point[0], point[1]) - f(point[0] + t*d[0], point[1] + t*d[1])) < (-alpha*t*np.dot(gradient_f(point[0], point[1]), d))
           and (2*(point[0] + t*d[0]) > (point[1] + t*d[1])) and (2*(point[1] + t*d[1]) > (point[0] + t*d[0])) and (point[0] + t*d[0] + point[1] + t*d[1] < 1)): 
        t = beta*t
    return t

- We implement the Combination Descent Algorithm below.
- First, we have a function `is_pos_def` which checks if a matrix is positive definite by seeing if all its eigenvalues are positive. 
- This works for a symmetric matrix and we can do this test on the Hessian Matrix $Hf(x)$ because the Hessian Matrix is always symmetric as $\frac{\partial^{2} f}{\partial x_1x_2} = \frac{\partial^{2} f}{\partial x_2x_1}$ (in the context of $R^2$)

- If our matrix is positive definite, we choose our $d = (Hf(x))^{-1}(-\nabla f(x_1,x_2))$ or the $d = -\nabla f(x_1,x_2)$
- Then we apply the Linear Search method to the step-size $t$ which we then use with the desent direction $d$ to update our x and y coordinates.
- We perform this in a loop till the value of $\nabla f(x) \leq \eta $ where $\eta$ is the tolerance level set by us.

In [11]:
# Function to check if a matrix is positive definite
def is_pos_def(x):

    # For checking if a matrix is positive definite we check if all the eigenvalues are positive (applicable for symmetric matrices)
    return np.all(np.linalg.eigvals(x) > 0)


# Implementing the Combinaion Descent Method
def combination_descent(x0, y0, alpha, beta, epsilon, max_iters = 300):
    # Initialize the variables
    x = x0
    y = y0

    #Initializing the loop
    while (np.linalg.norm(gradient_f(x,y)) > epsilon) and (max_iters != 0):
        
        # Compute the gradient
        grad = gradient_f(x,y)

        # Compute the Hessian
        hess = hessian_f(x,y)

        # Combination Descent Condition:
        if(is_pos_def(hess)):

            # If the Hessian is positive definite, we use the Newton Step
            
            # Computing the inverse of the Hessian
            hess_inverse = np.linalg.inv(hess)

            # Computing the Newton Step by taking the negative of the Hessian inverse dotted with the gradient vector
            d = -np.dot(hess_inverse, grad)
        
        else:
            
            # If the Hessian is not positive definite, we use the Gradient Descent Step which is the negative of the gradient vector
            d = -grad

        # Compute the step size
        t = linear_search(d, alpha, beta, [x,y])

        # Update the variables
        x = x + t*d[0]
        y = y + t*d[1]

        # Update the number of iterations
        max_iters = max_iters - 1

    # Return the solution as a numpy array with the number of iterations it took and the coordinates of the solution point
    return np.array([x, y, max_iters])



- Here we define the following values:
$$
(x_1,x_2) = (0.25, 0.25)
$$
$$
\alpha = 0.1
$$
$$
\beta = 0.3
$$
$$
\eta = 10^{-7}
$$
- We then start the combination descent algorithm

In [12]:
# Define the initial point
x0 = 0.25
y0 = 0.25

# Defining Constants
alpha = 0.2
beta = 0.5
epsilon = 1e-6

# Calling the function
point = combination_descent(x0, y0, alpha, beta, epsilon)

# Printing the results
print("The analytic center of S is at: (%.4f , %.4f)" %(point[0], point[1]))
print("The approximate minimum value of the potential function at the analytical center is: %.4f" %(f(point[0], point[1])))

The analytic center of S is at: (0.3333 , 0.3333)
The approximate minimum value of the potential function at the analytical center is: 3.2958


- We get the analytical center for the given potential function approximately for $f(x_1,x_2)$ at $(0.3333, 0.3333)$.
- The approximate minimum value of the potential function hence is $3.296$